In [6]:
from dotenv import load_dotenv
load_dotenv()
import os 


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### Using Wikipedia tool

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name

'wikipedia'

Create a Tool for Getting information from langSmith documentation 

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet star

In [10]:
vector_db = FAISS.from_documents(documents, OpenAIEmbeddings())
vector_db

In [11]:
retreiver = vector_db.as_retriever()
retreiver

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025951541A50>, search_kwargs={})

In [24]:
from langchain.tools.retriever import create_retriever_tool
langsmith_retreiver = create_retriever_tool(retreiver, "Langsmith_search", "Search for the information realted to LangSmith. For any question reslated to LangSmith, use this")

In [25]:
langsmith_retreiver.name

'Langsmith_search'

Use Arxiv Tool 

In [26]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(arxiv_wrapper = arxiv_wrapper)

In [27]:
arxiv.name

'arxiv'

Combining all the tools together 

In [28]:
tools = [wiki, langsmith_retreiver, arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\ML Projects\\Autogen-AgentAI\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Langsmith_search', description='Search for the information realted to LangSmith. For any question reslated to LangSmith, use this', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000259441AA4D0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025951541A50>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000259445083

In [29]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature=0)

In [30]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Creating a Agent, that combines all the tools 

In [31]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [32]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [33]:
agent_executor.invoke({"input": "What is langsmith?"})



> Entering new AgentExecutor chain...

Invoking: `Langsmith_search` with `{'query': 'What is Langsmith?'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - 

{'input': 'What is langsmith?',
 'output': "LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith provides observability features, evaluation capabilities, and tools for prompt engineering to help you optimize and improve your LLM applications. It is framework-agnostic and can be used with or without LangChain's open source frameworks langchain and langgraph. LangSmith's observability features help in debugging non-deterministic LLM applications and provide insights throughout all stages of application development."}

In [34]:
agent_executor.invoke({"input": "What is explained in the given paper 1705.05211"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1705.05211'}`


Published: 2018-01-25
Title: On-Grid DOA Estimation Method Using Orthogonal Matching Pursuit
Authors: Abhishek Aich, P. Palanisamy
Summary: Direction of Arrival (DOA) estimation of multiple narrow-band coherent or
partially coherent sources is a major challenge in array signal processing.
Though many subspace- based algorithms are available in literature, none of
them tackle the problem of resolving coherent sources directly, e.g. without
modifying the sample data covariance matrix. Compressive Sensing (CS) based
sparse recovery algorithms are being applied as a novel technique to this area.
In this paper, we introduce Orthogonal Matching Pursuit (OMP) to the DOA
estimation problem. We demonstrate how a DOA estimation problem can be modelled
for sparse recovery problem and then exploited using OMP to obtain the set of
DOAs. Moreover, this algorithm uses only one snapshot to obtain the results.
T

{'input': 'What is explained in the given paper 1705.05211',
 'output': 'The paper with identifier 1705.05211 is titled "On-Grid DOA Estimation Method Using Orthogonal Matching Pursuit" by Abhishek Aich and P. Palanisamy. The paper addresses the challenge of Direction of Arrival (DOA) estimation of multiple narrow-band coherent or partially coherent sources in array signal processing. It introduces the use of Orthogonal Matching Pursuit (OMP) to tackle the DOA estimation problem without modifying the sample data covariance matrix. The paper demonstrates how a DOA estimation problem can be modeled for sparse recovery and then exploited using OMP to obtain the set of DOAs. The algorithm presented in the paper requires only one snapshot to achieve results and shows advantages over existing subspace-based algorithms according to simulation results.'}

In [35]:
agent_executor.invoke({"input": "Tell me about Tom Cruise"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Tom Cruise'}`


Page: Tom Cruise
Summary: Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. Regarded as a Hollywood icon, he has received various accolades, including an Honorary Palme Thomas Cruise Mapother IV, known as Tom Cruise, is an American actor and producer. He was born on July 3, 1962. Tom Cruise is considered a Hollywood icon and has received numerous accolades for his work in the entertainment industry.

> Finished chain.


{'input': 'Tell me about Tom Cruise',
 'output': 'Thomas Cruise Mapother IV, known as Tom Cruise, is an American actor and producer. He was born on July 3, 1962. Tom Cruise is considered a Hollywood icon and has received numerous accolades for his work in the entertainment industry.'}